In [1]:
import shutil, os, csv, itertools, glob

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from dataloader import train_loader, val_loader

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading data...
Done
Loading data...
Done


In [7]:
%load_ext autoreload
%autoreload 1
%aimport configs
cfgs = configs
cuda = cfgs.USE_CUDA
%aimport model
CAN = model.CAN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def get_acc(output, target):
    # takes in two tensors to compute accuracy
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct = pred.eq(target.data.view_as(pred)).cpu().sum()
#     print("Output: ")
#     print(output.data.squeeze().cpu().numpy())
#     print("Pred: ")
#     print(pred.squeeze().cpu().numpy())
#     print("Target: ")
#     print(target.data.cpu().numpy())
    return correct, target.size(0)

def run_trainer(model_path, model, train_loader, test_loader, get_acc, resume, num_epoch):

    if not os.path.exists(model_path):
        os.makedirs(model_path)
    def save_checkpoint(state, is_best, filename=model_path+'checkpoint.pth.tar'):
        torch.save(state, filename)
        if is_best:
            shutil.copyfile(filename, model_path+'model_best.pth.tar')
    def get_last_checkpoint(model_path):
        fs = sorted([f for f in os.listdir(model_path) if 'Epoch' in f], key=lambda k: int(k.split()[1]))
        return model_path+fs[-1] if len(fs) > 0 else None
    
    start_epoch = 0
    best_res = 0
    resume_state = get_last_checkpoint(model_path) if resume else None
    if resume_state and os.path.isfile(resume_state):
        print("=> loading checkpoint '{}'".format(resume_state))
        checkpoint = torch.load(resume_state)
        start_epoch = checkpoint['epoch']+1
        best_res = checkpoint['val_acc']
        model.load_state_dict(checkpoint['state_dict'])
        if cuda:
            model.cuda()
        optimizer = optim.Adam(model.parameters(), **cfgs.OPT_PARAM)
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resume_state, checkpoint['epoch']))
    else:
        if cuda:
            model.cuda()
        optimizer = optim.Adam(model.parameters(), **cfgs.OPT_PARAM)

    criterion = nn.CrossEntropyLoss()
    # scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5) # optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5)

    def train(epoch):
        model.train()
        total, total_correct = 0., 0.
        for batch_idx, (img_feats, question, answer) in enumerate(train_loader):
            img_feats, question, answer = Variable(img_feats.float()), Variable(question.float()), Variable(answer.long())
            if cuda:
                img_feats, question, answer = img_feats.cuda(), question.cuda(), answer.cuda()
            optimizer.zero_grad()
            output = model(img_feats, question)
            loss = criterion(output, answer)
            loss.backward()
            optimizer.step()

            correct, num_instance = get_acc(output, answer)
            total_correct += correct
            total += num_instance
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} Acc: {:.2f}%/{:.2f}%'.format(
                    epoch, batch_idx * cfgs.BATCH_SIZE, len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0],
                    100. * correct / num_instance, 100. * total_correct / total ))
        
        return 100. * total_correct / total

    def test():
        model.eval()
        test_loss = 0.
        total, total_correct = 0., 0.
        for img_feats, question, answer in test_loader:
            img_feats, question, answer = Variable(img_feats.float()), Variable(question.float()), Variable(answer.long())
            if cuda:
                img_feats, question, answer = img_feats.cuda(), question.cuda(), answer.cuda()
            optimizer.zero_grad()
            output = model(img_feats, question)
            test_loss += criterion(output, answer).data[0] # sum up batch loss
            
            correct, num_instance = get_acc(output, answer)
            total_correct += correct
            total += num_instance

        test_acc = 100. * total_correct / total
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, total_correct, total,
            test_acc))

        return test_acc


    for epoch in range(start_epoch, num_epoch):
        is_best = False

        train_acc = train(epoch)
        val_acc = test()
        
        # scheduler.step(val_loss)

        if val_acc > best_res:
            best_res = val_acc
            is_best = True

        save_checkpoint({
                'epoch': epoch,
                'state_dict': model.cpu().state_dict(),
                'train_acc':train_acc,
                'val_acc': val_acc,
                'optimizer' : optimizer.state_dict(),
            }, is_best,
            model_path+"Epoch %d Acc %.4f.pt"%(epoch, val_acc))

        if cuda:
            model.cuda()

In [ ]:
model = CAN(**cfgs.NET_PARAM)
run_trainer(
    model_path = './ckpt/', 
    model = model, 
    train_loader = train_loader, 
    test_loader = val_loader, 
    get_acc = get_acc, 
    resume = False, 
    num_epoch = 100
)

Train Epoch: 0 [0/699989 (0%)]	Loss: 3.321915 Acc: 1.56%/1.56%
Train Epoch: 0 [640/699989 (0%)]	Loss: 2.982973 Acc: 17.19%/20.31%
Train Epoch: 0 [1280/699989 (0%)]	Loss: 2.663429 Acc: 26.56%/20.68%
Train Epoch: 0 [1920/699989 (0%)]	Loss: 2.758787 Acc: 17.19%/20.16%
Train Epoch: 0 [2560/699989 (0%)]	Loss: 2.958366 Acc: 15.62%/20.24%
Train Epoch: 0 [3200/699989 (0%)]	Loss: 2.451110 Acc: 28.12%/20.56%
Train Epoch: 0 [3840/699989 (1%)]	Loss: 2.729137 Acc: 14.06%/20.44%
Train Epoch: 0 [4480/699989 (1%)]	Loss: 2.586968 Acc: 25.00%/20.97%
Train Epoch: 0 [5120/699989 (1%)]	Loss: 2.435516 Acc: 23.44%/21.47%
Train Epoch: 0 [5760/699989 (1%)]	Loss: 2.087361 Acc: 31.25%/22.39%
Train Epoch: 0 [6400/699989 (1%)]	Loss: 1.899316 Acc: 23.44%/22.94%
Train Epoch: 0 [7040/699989 (1%)]	Loss: 1.971354 Acc: 21.88%/23.62%
Train Epoch: 0 [7680/699989 (1%)]	Loss: 1.841437 Acc: 26.56%/23.93%
Train Epoch: 0 [8320/699989 (1%)]	Loss: 2.000602 Acc: 17.19%/24.42%
Train Epoch: 0 [8960/699989 (1%)]	Loss: 1.953207 Acc: 

Train Epoch: 0 [76160/699989 (11%)]	Loss: 0.933280 Acc: 42.19%/37.82%
Train Epoch: 0 [76800/699989 (11%)]	Loss: 0.883422 Acc: 45.31%/37.86%
Train Epoch: 0 [77440/699989 (11%)]	Loss: 0.999142 Acc: 37.50%/37.89%
Train Epoch: 0 [78080/699989 (11%)]	Loss: 1.052877 Acc: 39.06%/37.94%
Train Epoch: 0 [78720/699989 (11%)]	Loss: 0.910115 Acc: 37.50%/37.97%
Train Epoch: 0 [79360/699989 (11%)]	Loss: 1.098190 Acc: 40.62%/38.00%
Train Epoch: 0 [80000/699989 (11%)]	Loss: 1.070198 Acc: 42.19%/38.05%
Train Epoch: 0 [80640/699989 (12%)]	Loss: 0.964588 Acc: 54.69%/38.09%
Train Epoch: 0 [81280/699989 (12%)]	Loss: 0.992613 Acc: 53.12%/38.15%
Train Epoch: 0 [81920/699989 (12%)]	Loss: 1.222030 Acc: 39.06%/38.21%
Train Epoch: 0 [82560/699989 (12%)]	Loss: 1.010980 Acc: 53.12%/38.24%
Train Epoch: 0 [83200/699989 (12%)]	Loss: 1.076584 Acc: 35.94%/38.29%
Train Epoch: 0 [83840/699989 (12%)]	Loss: 0.920271 Acc: 50.00%/38.35%
Train Epoch: 0 [84480/699989 (12%)]	Loss: 1.152637 Acc: 32.81%/38.37%
Train Epoch: 0 [8512

Train Epoch: 0 [150400/699989 (21%)]	Loss: 0.901690 Acc: 45.31%/40.90%
Train Epoch: 0 [151040/699989 (22%)]	Loss: 0.977090 Acc: 48.44%/40.91%
Train Epoch: 0 [151680/699989 (22%)]	Loss: 1.023801 Acc: 34.38%/40.94%
Train Epoch: 0 [152320/699989 (22%)]	Loss: 1.000446 Acc: 45.31%/40.96%
Train Epoch: 0 [152960/699989 (22%)]	Loss: 0.967268 Acc: 54.69%/40.98%
Train Epoch: 0 [153600/699989 (22%)]	Loss: 0.971132 Acc: 43.75%/41.01%
Train Epoch: 0 [154240/699989 (22%)]	Loss: 0.935738 Acc: 46.88%/41.03%
Train Epoch: 0 [154880/699989 (22%)]	Loss: 0.912891 Acc: 50.00%/41.06%
Train Epoch: 0 [155520/699989 (22%)]	Loss: 0.918632 Acc: 37.50%/41.09%
Train Epoch: 0 [156160/699989 (22%)]	Loss: 0.914777 Acc: 46.88%/41.11%
Train Epoch: 0 [156800/699989 (22%)]	Loss: 1.067858 Acc: 45.31%/41.12%
Train Epoch: 0 [157440/699989 (22%)]	Loss: 1.108058 Acc: 40.62%/41.13%
Train Epoch: 0 [158080/699989 (23%)]	Loss: 1.046415 Acc: 42.19%/41.14%
Train Epoch: 0 [158720/699989 (23%)]	Loss: 0.982188 Acc: 43.75%/41.16%
Train 